In [1]:
!pip install polars beautifulsoup4 splinter selenium scikit-learn altair matplotlib tensorflow imbalanced-learn

In [2]:
#Data Manipulation and display toools
import polars as pl
from polars.exceptions import InvalidOperationError
import altair as alt
import numpy as np
import pathlib
# Pandas is necessary for ease of use with the ML libraries
# Polars does not use indexing and therefore does not the following data format properly
#   {tablename: {index1: [row1_values]},
#               {index2: [row2_values]}}
import pandas as pd
from IPython.display import display_html 
#Necessary for tensorflow on my machine due to distutils being depreciated
import setuptools

# Web scraping tools
import re
from bs4 import BeautifulSoup
# Uses firefox browser
from splinter import Browser
import time

# Data preprocessing tools
from sklearn.preprocessing import StandardScaler, OneHotEncoder, KBinsDiscretizer
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.decomposition import PCA
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import make_pipeline

# Machine learning tools
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.metrics import classification_report


from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Sequential

random_state = 2112250415

2024-10-10 12:44:10.304625: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-10 12:44:10.305091: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-10-10 12:44:10.307168: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-10-10 12:44:10.312501: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-10 12:44:10.320805: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been 

# Loading Data

In [3]:
cwd = pathlib.Path.cwd()

if cwd.name == 'Mild-Steel-Tempering':
    print("Path is project root")
else:
    print("Please correct current working directory to the project root")


Path is project root


In [4]:
resources_path = pathlib.PurePath(pathlib.PurePath(cwd), 'resources')
resources_path

PurePosixPath('/home/mox/Documents/coding_projects/bootcamp_local/Homeworks/Mild-Steel-Tempering/resources')

### Many alloy composition columns were parsed incorrectly and failing to load

Several weight percent columns were parsed as int automatically due to having "0" for many initial rows. 

All weight percent columns should be parsed as float. 

In [5]:
data_path = f"{resources_path}/Raiipa-tempering-data.csv"
schema_overrides = {"C (%wt)" : pl.Float64,
"Mn (%wt)" : pl.Float64,
"P (%wt)" : pl.Float64,
"S (%wt)" : pl.Float64,
"Si (%wt)" : pl.Float64,
"Ni (%wt)" : pl.Float64,
"Cr (%wt)" : pl.Float64,
"Mo (%wt)" : pl.Float64,
"V (%wt)" : pl.Float64,
"Al (%wt)" : pl.Float64,
"Cu (%wt)" : pl.Float64}

df_data = pl.read_csv(data_path, schema_overrides=schema_overrides)
df_data.glimpse()

Rows: 1466
Columns: 17
$ Source                                  <str> 'Grange and Baughman, 1956', 'Grange and Baughman, 1956', 'Grange and Baughman, 1956', 'Grange and Baughman, 1956', 'Grange and Baughman, 1956', 'Grange and Baughman, 1956', 'Grange and Baughman, 1956', 'Grange and Baughman, 1956', 'Grange and Baughman, 1956', 'Grange and Baughman, 1956'
$ Steel type                              <str> 'AISI-SAE 1026', 'AISI-SAE 1026', 'AISI-SAE 1026', 'AISI-SAE 1026', 'AISI-SAE 1026', 'AISI-SAE 1026', 'AISI-SAE 1026', 'AISI-SAE 1026', 'AISI-SAE 1026', 'AISI-SAE 1026'
$ Initial hardness (HRC) - post quenching <str> '?', '?', '?', '?', '?', '?', '?', '?', '?', '?'
$ Tempering time (s)                      <i64> 600, 600, 600, 600, 600, 600, 600, 600, 600, 600
$ Tempering temperature (ºC)              <f64> 204.4, 260.0, 315.6, 371.1, 426.7, 482.2, 537.8, 593.3, 648.9, 704.4
$ C (%wt)                                 <f64> 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25
$ Mn 

In [6]:
# save initial columns names as they are very descriptive and may be useful later
initial_column_names = df_data.columns
initial_column_names

['Source',
 'Steel type',
 'Initial hardness (HRC) - post quenching',
 'Tempering time (s)',
 'Tempering temperature (ºC)',
 'C (%wt)',
 'Mn (%wt)',
 'P (%wt)',
 'S (%wt)',
 'Si (%wt)',
 'Ni (%wt)',
 'Cr (%wt)',
 'Mo (%wt)',
 'V (%wt)',
 'Al (%wt)',
 'Cu (%wt)',
 'Final hardness (HRC) - post tempering']

# Cleaning
Many columns need renaming for ease of manipulation

Columns need datatypes correction
 'Initial hardness (HRC) - post quenching' needs datatype correction. ? is NA value


In [7]:
dict_new_cnames = {}
for og_name in initial_column_names:
    # replace filler in hardness columns
    new_name = og_name.replace(' - ', '')
    
    # Handle units
    try:
        #if alloy weight percent remove units and return only elemental symbol
        if re.search(r"\(%wt\)", og_name):
            new_name = og_name.split(' ')[0]
            #skip the rest of the try block that will re_add the units to the end
            pass
        else: 
            #If not elemental composition, lowercase the string
            new_name = new_name.lower()
        # regex find the units inside the parenthesis, of the original name, not the lowercased new name
        # This lines breaks and goes to except if there is no units
        unit = re.search(r'\((\w+)\)', og_name).group(1)
        # replace the unit parenthesis string with parenthesis with an empty string
        new_name = re.sub(r"\(.+\)", "", new_name)
        # trim to whitespace end characters left by some unit removals
        new_name = new_name.rstrip()
        # append the unit string to the end of the processed name
        new_name = f"{new_name}_{unit}"
    except:
        # skip unit processing on names with no units denoted by parenthesis
        pass

    # Strip away special characters
    new_name = new_name.encode("ascii", errors="ignore").decode()
    #replace all whitespace with underscores
    new_name = new_name.replace(' ', '_')
    #add the name to the rename dict
    dict_new_cnames[og_name] = new_name
dict_new_cnames


{'Source': 'source',
 'Steel type': 'steel_type',
 'Initial hardness (HRC) - post quenching': 'initial_hardness_post_quenching_HRC',
 'Tempering time (s)': 'tempering_time_s',
 'Tempering temperature (ºC)': 'tempering_temperature_C',
 'C (%wt)': 'C',
 'Mn (%wt)': 'Mn',
 'P (%wt)': 'P',
 'S (%wt)': 'S',
 'Si (%wt)': 'Si',
 'Ni (%wt)': 'Ni',
 'Cr (%wt)': 'Cr',
 'Mo (%wt)': 'Mo',
 'V (%wt)': 'V',
 'Al (%wt)': 'Al',
 'Cu (%wt)': 'Cu',
 'Final hardness (HRC) - post tempering': 'final_hardness_post_tempering_HRC'}

In [8]:
df_clean_cnames = df_data.rename(dict_new_cnames)

## Clean data types and column values

In [9]:
count_of_qmark = df_clean_cnames['initial_hardness_post_quenching_HRC'].value_counts()\
    .filter(pl.col('initial_hardness_post_quenching_HRC') == "?")\
        .select("count").item()


In [10]:
percent_intial_hardness_unknown = count_of_qmark / int(df_clean_cnames['initial_hardness_post_quenching_HRC'].shape[0])
print(f"Unknown initial hardness: {round(percent_intial_hardness_unknown, 2) * 100:.0f}%")


Unknown initial hardness: 65%


In [11]:
df_clean_cnames.tail(3)


source,steel_type,initial_hardness_post_quenching_HRC,tempering_time_s,tempering_temperature_C,C,Mn,P,S,Si,Ni,Cr,Mo,V,Al,Cu,final_hardness_post_tempering_HRC
str,str,str,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""Hollomon and Jaffe, 1945""","""1,15%C - plain carbon steel""","""64.5""",86400,500.0,1.15,0.58,0.012,0.021,0.09,0.0,0.01,0.0,0.0,0.0,0.0,32.0
"""Hollomon and Jaffe, 1945""","""1,15%C - plain carbon steel""","""64.5""",86400,600.0,1.15,0.58,0.012,0.021,0.09,0.0,0.01,0.0,0.0,0.0,0.0,23.0
"""Hollomon and Jaffe, 1945""","""1,15%C - plain carbon steel""","""64.5""",86400,700.0,1.15,0.58,0.012,0.021,0.09,0.0,0.01,0.0,0.0,0.0,0.0,4.5


In [12]:
df_clean_cnames["initial_hardness_post_quenching_HRC"].value_counts().sort('count', descending=True).head(3)

initial_hardness_post_quenching_HRC,count
str,u32
"""?""",949
"""66.5""",90
"""55.8""",51


In [13]:
df_clean = df_clean_cnames.with_columns(pl.col('initial_hardness_post_quenching_HRC').cast(pl.Float64, strict=False))


In [14]:
df_clean["initial_hardness_post_quenching_HRC"].value_counts().sort('count', descending=True).head(3)

initial_hardness_post_quenching_HRC,count
f64,u32
null,949
66.5,90
55.8,51


In [15]:
df_clean.describe()

statistic,source,steel_type,initial_hardness_post_quenching_HRC,tempering_time_s,tempering_temperature_C,C,Mn,P,S,Si,Ni,Cr,Mo,V,Al,Cu,final_hardness_post_tempering_HRC
str,str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""count""","""1466""","""1466""",517.0,1466.0,1466.0,1466.0,1466.0,1466.0,1466.0,1466.0,1466.0,1466.0,1466.0,1466.0,1466.0,1466.0,1466.0
"""null_count""","""0""","""0""",949.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"""mean""",null,null,61.493617,21969.754434,422.024147,0.511583,0.74073,0.017236,0.023802,0.239379,0.362838,0.389696,0.080232,0.005457,0.034379,0.005986,41.468008
"""std""",null,null,5.656383,34177.623863,176.088041,0.224354,0.252913,0.007966,0.007967,0.239193,0.810091,0.480721,0.121422,0.02905,0.20534,0.019085,14.079248
"""min""","""Grange and Baughman, 1956""","""0,31%C - plain carbon steel""",46.5,10.0,100.0,0.25,0.3,0.007,0.005,0.06,0.0,0.0,0.0,0.0,0.0,0.0,0.9
"""25%""",null,null,58.8,600.0,260.0,0.37,0.6,0.012,0.018,0.16,0.0,0.02,0.0,0.0,0.0,0.0,32.1
"""50%""",null,null,63.1,3600.0,426.7,0.42,0.74,0.017,0.024,0.21,0.01,0.06,0.0,0.0,0.0,0.0,43.1
"""75%""",null,null,66.5,14400.0,593.3,0.56,0.8,0.019,0.029,0.24,0.06,0.8,0.22,0.0,0.0,0.0,51.8
"""max""","""Penha, 2010""","""Nitriding Steel """,67.0,115200.0,704.4,1.15,1.85,0.054,0.055,1.62,3.41,1.57,0.36,0.16,1.26,0.08,68.5


# Making steel types values searchable on AZoM.com
Steel types that do not include AISI and a code are not searchable in steel databased and need manual renaming

Exmaple: "0,74%C - plain carbon steel" is a AISI 1074 steel. Confirmed via elemental composition on https://www.azom.com/article.aspx?ArticleID=6558


In [16]:
steel_identifiers = ["steel_type", "source"]
df_clean[steel_identifiers].unique().shape

(36, 2)

## Renaming all possible steels so they are searchable

1945 source

I can not access source papers. Making a best guess of steel grade from elemental composition

I assume some alloying elements are a bit strange due to wartime shortages and potential contamination between alloys due to the push to increasew prodiction.

### additional Justifications are inline

In [17]:
#1945 source...
# I can not access source for the 1945 paper on these
# I assume they alloying elements are a bit strange due to wartime shortages
# and potentially contamination between alloys due to the push to increasew prodiction
dict_rename = {'0,98%C - plain carbon steel': 'AISI 1095', # copper and Cr, probably intentional?
                   '1,15%C - plain carbon steel': 'AISI 1095', # small chromium impurity?
                   '0,74%C - plain carbon steel': 'AISI 1074 Carbon Steel', 
                   '0,56%C - plain carbon steel': 'AISI 1055', # small chromium impurity?
                   '0,89%C - plain carbon steel': 'AISI 1090', # Cu discounted as impurity
                   'Nitriding Steel ': 'Non-searchable', 
                   '0,31%C - plain carbon steel': 'AISI 1030', # AISI 1030, Cr impurity
                   "AISI-SAE 9264" : 'AISI-SAE 9254', # access limited, may be AISI-SAE 9264 , not in AZoM
                   "AISI-SAE 2340" : 'Non-searchable', # access limited, may be SAE J2340, not in AZoM
                   "AISI-SAE 3140" : 'Non-searchable', # Not in AZoM elemental match to SAE 3140 https://www.steel-grades.com/metals/18/5155/-SAE-3140.html
                   "AISI-SAE 4068" : 'Non-searchable', # access limited, may be SAE 4068, not in AZoM
                   "AISI-SAE 4640" : "AISI 4640",
                   "AISI-SAE 4047" : "AISI 4047",
                   "AISI-SAE 1049" : "AISI 1049",
                   "AISI-SAE 6145" : "AISI 6145",
                   "AISI-SAE E52100" : "AISI 52100"} # # Not in AZoM elemental match to  SAE 4068https://www.steel-grades.com/Steel-Grades/Carbon-Steel/SAE-4068-.html

In [18]:
series_searchable_steel = df_clean.clone()
renamed = series_searchable_steel['steel_type'].replace(dict_rename)
series_searchable_steel = series_searchable_steel.with_columns(searchable = renamed)

In [19]:
series_to_search = series_searchable_steel['searchable'].unique()
steels_to_search = series_to_search.filter(series_to_search.eq('Non-searchable').not_())
steels_to_search

searchable
str
"""AISI-SAE 1038"""
"""AISI-SAE 1065"""
"""AISI-SAE 5160"""
"""AISI-SAE 1080"""
"""AISI 1090"""
…
"""AISI-SAE 1050"""
"""AISI-SAE 4340"""
"""AISI-SAE 4037"""


## Save Searchable steel
Checkpoint so that the code to this point does not need to be run every time

In [20]:
path_save_searchable = cwd / 'resources' / 'searchable_steels.csv'
pl.DataFrame(steels_to_search).write_csv(path_save_searchable)

# Web Scraping

Two Stages are necessary

AZoM does not have data on all steels.

MakeItFrom.com also does not have data from all steels. 

TODO: together they have data from ?? steels

In [21]:
len([print(steel) for steel in steels_to_search])

AISI-SAE 1038
AISI-SAE 1065
AISI-SAE 5160
AISI-SAE 1080
AISI 1090
AISI 1055
AISI 1030
AISI-SAE 5140
AISI-SAE 4140
AISI-SAE 1042
AISI 4640
AISI-SAE 1045
AISI 1049
AISI 1074 Carbon Steel
AISI-SAE 9254
AISI 4047
AISI-SAE 1026
AISI-SAE 1040
AISI 52100
AISI-SAE 4027
AISI-SAE 1030
AISI 1095
AISI-SAE 1035
AISI-SAE 1335
AISI-SAE 1050
AISI-SAE 4340
AISI-SAE 4037
AISI-SAE 6150
AISI 6145


29

## Scraping AZoM.com

In [22]:
browser = Browser('firefox')
base_url = "https://www.azom.com"
search_path = "/search.aspx?q="

In [23]:
# Search the steel 
def get_soup(soup):
    search_result = soup.find('div', class_='resultsContainer')
    first_item = search_result.find('a')
    # https://pytutorial.com/get-element-href-beautifulsoup/
    steel_link = first_item.get('href')
    time.sleep(0.15)
    browser.visit(f"{base_url}{steel_link}")
    soup2 = BeautifulSoup(browser.html, 'html.parser')
    return soup2


In [24]:
# already have elemental composition
# property - metric - imperial
# table_elements = tables[0]
# table_phys_props = tables[1]
# table_mecha_props = tables[2]
# table_therm_props = tables[3]
# table_other_desigs = tables[4]
#parse tables
def get_tables(soup):
    tables = []
    # parse all tables in the
    for html_table in soup.find_all('table'):
        table = []
        # parse rows in the table
        for tr in html_table.find_all('tr'):
            row = []
            #parse data cells in the row
            for t in tr.find_all(['th', 'td']):
                text = t.get_text(strip=True)
                #add data to row
                row.append(text)
            #add row to table
            table.append(row)
    #returns a list of list of lists
        tables.append(table)
    return tables
    

In [25]:
def make_dfs(lolols):
    tables = []
    # convert the list of lists to a dataframe
    for lol in lolols:
        try:
            df = pl.DataFrame(lol, orient='row')
            # rename columns using first row
            # https://stackoverflow.com/questions/75187317/how-to-rename-column-names-with-first-row-in-polars
            # [1:] removes the first row that replaced the column names 
            df = df.rename(df.head(1).to_dicts().pop())[1:]
            # add the dataframe to the list of tables on for this steel
            tables.append(df)
        except:
            tables.append(lol)
    return tables

In [26]:
dict_steel_tables = {}
for steel in steels_to_search:
    try:
        print("Working on:", steel)
        #search AZoM for the steel
        browser.visit(f"{base_url}{search_path}{steel}")
        soup_search = BeautifulSoup(browser.html, 'html.parser')
    
        #takes soup html and returns list of lists
        soup = get_soup(soup_search)
        lolols = get_tables(soup)
        dict_steel_tables[steel] = make_dfs(lolols)
        # tables list of lists and returns a dictionary
        # robots.txt indicates Crawl-delay: 120
        time.sleep(0.15)
    except: 
        dict_steel_tables[steel] = []

Working on: AISI-SAE 1038
Working on: AISI-SAE 1065
Working on: AISI-SAE 5160
Working on: AISI-SAE 1080
Working on: AISI 1090
Working on: AISI 1055
Working on: AISI 1030
Working on: AISI-SAE 5140
Working on: AISI-SAE 4140
Working on: AISI-SAE 1042
Working on: AISI 4640
Working on: AISI-SAE 1045
Working on: AISI 1049
Working on: AISI 1074 Carbon Steel
Working on: AISI-SAE 9254
Working on: AISI 4047
Working on: AISI-SAE 1026
Working on: AISI-SAE 1040
Working on: AISI 52100
Working on: AISI-SAE 4027
Working on: AISI-SAE 1030
Working on: AISI 1095
Working on: AISI-SAE 1035
Working on: AISI-SAE 1335
Working on: AISI-SAE 1050
Working on: AISI-SAE 4340
Working on: AISI-SAE 4037
Working on: AISI-SAE 6150
Working on: AISI 6145


In [27]:
dict_steel_tables

{'AISI-SAE 1038': [shape: (5, 2)
  ┌────────────────┬─────────────┐
  │ Element        ┆ Content (%) │
  │ ---            ┆ ---         │
  │ str            ┆ str         │
  ╞════════════════╪═════════════╡
  │ Iron, Fe       ┆ 98.59-99.09 │
  │ Manganese, Mn  ┆ 0.60-0.90   │
  │ Carbon, C      ┆ 0.340-0.420 │
  │ Sulfur, S      ┆ ≤ 0.050     │
  │ Phosphorous, P ┆ ≤ 0.040     │
  └────────────────┴─────────────┘,
  shape: (1, 3)
  ┌────────────┬─────────────┬───────────────┐
  │ Properties ┆ Metric      ┆ Imperial      │
  │ ---        ┆ ---         ┆ ---           │
  │ str        ┆ str         ┆ str           │
  ╞════════════╪═════════════╪═══════════════╡
  │ Density    ┆ 7.845 g/cm3 ┆ 0.2834 lb/in³ │
  └────────────┴─────────────┴───────────────┘,
  shape: (13, 3)
  ┌─────────────────────────────────┬─────────────┬─────────────────┐
  │ Properties                      ┆ Metric      ┆ Imperial        │
  │ ---                             ┆ ---         ┆ ---             │
  │ str 

# Converting AZoM Scraped Data to one dataframe

## Working with list of tables

Must confirma all tables were sucesffuly made into polars tables

Must reduce the numebr of tables to only those we are concerned with

* Remove "Other Designations" tables, including those that did not get sucessfully converted to a polars dataframe (table 4)

* Remove tables with properties that are not commonly present (thermal propterties table 3)

* Remove elemental compositions, already present in the data (table 0)

* Physical and mechanical properties are desired (tables 1,2)

In [28]:
dict_steel_tables_keep = {}
bool_check = True
polars_check = True 
for steel, tables in dict_steel_tables.items():
    all_polars = True
    count_polars = 0
    for table in tables:
        if isinstance(table, pl.DataFrame): 
            count_polars += 1 
    if count_polars != len(tables):
        all_polars = False
    if not (bool_check and all_polars):
        bool_check = False
    print("All polars?", all_polars, '|', len(tables), "Tables | Steel:", steel)
    dict_steel_tables_keep[steel] = tables[1:3]
print(polars_check)

All polars? True | 4 Tables | Steel: AISI-SAE 1038
All polars? True | 4 Tables | Steel: AISI-SAE 1065
All polars? True | 4 Tables | Steel: AISI-SAE 5160
All polars? True | 4 Tables | Steel: AISI-SAE 1080
All polars? True | 4 Tables | Steel: AISI 1090
All polars? True | 4 Tables | Steel: AISI 1055
All polars? True | 5 Tables | Steel: AISI 1030
All polars? True | 5 Tables | Steel: AISI-SAE 5140
All polars? True | 5 Tables | Steel: AISI-SAE 4140
All polars? True | 4 Tables | Steel: AISI-SAE 1042
All polars? True | 4 Tables | Steel: AISI 4640
All polars? True | 5 Tables | Steel: AISI-SAE 1045
All polars? True | 3 Tables | Steel: AISI 1049
All polars? True | 3 Tables | Steel: AISI 1074 Carbon Steel
All polars? True | 0 Tables | Steel: AISI-SAE 9254
All polars? True | 4 Tables | Steel: AISI 4047
All polars? True | 5 Tables | Steel: AISI-SAE 1026
All polars? True | 4 Tables | Steel: AISI-SAE 1040
All polars? True | 4 Tables | Steel: AISI 52100
All polars? True | 4 Tables | Steel: AISI-SAE 402

### Check if all kept tables are good

Confirm all are polars dataframes

Look at the shape of all of them and determine which will be limiting

In [29]:
dict_phys_props = {}
dict_mech_props = {}

polars_check = True 
for steel, tables in dict_steel_tables_keep.items():
    try:
        all_polars = True
        count_polars = 0
        for table in tables:
            if isinstance(table, pl.DataFrame): 
                count_polars += 1 
        if count_polars != len(tables):
            all_polars = False
        if not (bool_check and all_polars):
            bool_check = False
        print("All polars?", all_polars, '|', len(tables), "Tables | Steel:", steel)

        phys_props = tables[0]
        mech_props = tables[1]
        print("Physical properties:", phys_props.shape, "| Properties head(1)", phys_props.select(pl.col('Properties').head(1)) )
        print("Mechanical propsshape:", mech_props.shape, "| Properties head(1)", mech_props.select(pl.col('Properties').head(1)) )
        dict_phys_props[steel] = phys_props
        dict_mech_props[steel] = mech_props
    except:
        dict_phys_props[steel] = pl.DataFrame({'Properties' : [], 'Metric' : []})
        dict_mech_props[steel] = pl.DataFrame({'Properties' : [], 'Metric' : []})

print("All Polars?", polars_check)

All polars? True | 2 Tables | Steel: AISI-SAE 1038
Physical properties: (1, 3) | Properties head(1) shape: (1, 1)
┌────────────┐
│ Properties │
│ ---        │
│ str        │
╞════════════╡
│ Density    │
└────────────┘
Mechanical propsshape: (13, 3) | Properties head(1) shape: (1, 1)
┌────────────────────────────┐
│ Properties                 │
│ ---                        │
│ str                        │
╞════════════════════════════╡
│ Tensile strength, ultimate │
└────────────────────────────┘
All polars? True | 2 Tables | Steel: AISI-SAE 1065
Physical properties: (1, 3) | Properties head(1) shape: (1, 1)
┌────────────┐
│ Properties │
│ ---        │
│ str        │
╞════════════╡
│ Density    │
└────────────┘
Mechanical propsshape: (14, 3) | Properties head(1) shape: (1, 1)
┌────────────────────────────┐
│ Properties                 │
│ ---                        │
│ str                        │
╞════════════════════════════╡
│ Tensile strength, ultimate │
└──────────────────────────

# Filter for complete data, and select unit standard

Kpep only property and metric column from both sets of tables

Find which properties are present in all tables of each type

In [30]:
def remove_imperial(property_dict):
    new_dict = {}
    for key, table in property_dict.items():
        new_dict[key] = table.select(["Properties", "Metric"])
    return new_dict

dict_phys_props =  remove_imperial(dict_phys_props)
dict_mech_props = remove_imperial(dict_mech_props)


dict_phys_props['AISI-SAE 1050']

Properties,Metric
str,str
"""Density""","""7.85 g/cm3"""


# Properties to keep

Only properties present in all metals are the following:

Physical
* Density

Mechanical
* Elastic modulus
* Poisson's ratio

## Process Density into a df

In [31]:
#fix formatting in single cell. Did not incude a space 
# TODO: handle with regex to additional similar errors in additional rows
# arrives as '7.7-8.03g/cm3'
dict_phys_props['AISI 1074 Carbon Steel'][0, "Metric"] = '7.7-8.03 g/cm3'
# arrives as '200GPa'
dict_mech_props['AISI-SAE 1065'][2, "Metric"] = '200 GPa'
dict_mech_props['AISI-SAE 1065']

Properties,Metric
str,str
"""Tensile strength, ultimate""","""635 MPa"""
"""Tensile strength, yield""","""490 MPa"""
"""Modulus of elasticity""","""200 GPa"""
"""Bulk modulus (typical for stee…","""140 GPa"""
"""Shear modulus (typical for ste…","""80 GPa"""
…,…
"""Hardness, Knoop (converted fro…","""209"""
"""Hardness, Rockwell B (converte…","""90"""
"""Hardness, Rockwell C (converte…","""10"""


In [32]:
s_steel = pl.Series(name = 'steel', dtype= pl.String)
s_density = pl.Series(name = 'density', dtype= pl.String)
s_unit = pl.Series(name = 'units_density', dtype= pl.String)

for steel, table in dict_phys_props.items():
    try:
        #select only the  cell of interest
        df_density = table.filter(pl.col('Properties').str.contains("Density"))['Metric'].str.split(' ')
        density = df_density.list.get(0)
        unit = df_density.list.get(1)
        # Make a series of all 3
        s_steel.extend(pl.Series(name = 'steel', values = [steel]))
        s_density = s_density.extend(density)
        s_unit = s_unit.extend(unit)
    except:
        print("Failed on", steel)

Failed on AISI-SAE 9254


In [33]:
df_steel_properties = pl.DataFrame([s_steel, s_density, s_unit])
df_steel_properties

steel,density,units_density
str,str,str
"""AISI-SAE 1038""","""7.845""","""g/cm3"""
"""AISI-SAE 1065""","""7.85""","""g/cm3"""
"""AISI-SAE 5160""","""7.85""","""g/cm3"""
"""AISI-SAE 1080""","""7.7-8.03""","""g/cm3"""
"""AISI 1090""","""7.85""","""g/cm3"""
…,…,…
"""AISI-SAE 1050""","""7.85""","""g/cm3"""
"""AISI-SAE 4340""","""7.85""","""g/cm3"""
"""AISI-SAE 4037""","""7.85""","""g/cm3"""


## Process mechanical properties

In [34]:
def grab_property(table, property):
    s_result = table.filter(pl.col('Properties')\
                            .str.to_lowercase()\
                                .str.contains(property))\
                                ['Metric']
    return s_result

In [35]:
def get_oobT(df, index):
    return df.list.get(index, null_on_oob=True)

In [36]:
# list of tuples if data was sucessfully scraped from AZoM
azom_sucessful = {'steel':[],'AZoM-Successful':[]}
list_of_scraped = [("steel", "pr", "em", "em_unit", "ys", "ys_unit", "uts", "uts_unit")]

for steel, table in dict_mech_props.items():
    pr = None
    em = None
    em_unit = None
    ys = None
    ys_unit = None
    uts = None
    uts_unit = None

    try:
        # Grab values and split into units where necessary
        #poisson's ratio
        pr = grab_property(table, "poisson")

        #elastic modulus
        df_em = grab_property(table, "elastic").str.split(' ')
        em = get_oobT(df_em, 0)
        em_unit = get_oobT(df_em, 1)
        # em = df_em.list.get(0, null_on_oob=True)
        # em_unit = df_em.list.get(1, null_on_oob=True)

        # grab yield strength
        df_ys = grab_property(table, "yield").str.split(' ')
        ys = get_oobT(df_ys, 0)
        ys_unit = get_oobT(df_ys, 1)

        # grab strength value that is not yield strength.
        # Tensile strength or ultimate yield strength or ultimate tensile strength. All the same measurement
        df_uts = table.filter(pl.col('Properties').str.to_lowercase().str.contains("strength") & 
                            pl.col('Properties').str.to_lowercase().str.contains("yield").not_())\
                                ['Metric'].str.split(' ')
        uts = get_oobT(df_uts, 0)
        uts_unit = get_oobT(df_uts, 1)
    
        # Make a tuple for each row
        # If this fails it will be an IndexError and go to the except
        row = (steel, pr[0], em[0], em_unit[0], ys[0], ys_unit[0], uts[0], uts_unit[0])

        list_of_scraped.append(row)
        azom_sucessful['steel'].append(steel)
        azom_sucessful['AZoM-Successful'].append(True)
        
    except (IndexError, InvalidOperationError):
        print("Failed on", steel)
        azom_sucessful['steel'].append(steel)
        azom_sucessful['AZoM-Successful'].append(False)

df_AZoM_successfull = pl.DataFrame(azom_sucessful)
df_AZoM_successfull.filter(pl.col('AZoM-Successful')==False)

Failed on AISI-SAE 1080
Failed on AISI-SAE 1042
Failed on AISI 1049
Failed on AISI-SAE 9254
Failed on AISI 4047
Failed on AISI 52100
Failed on AISI-SAE 1335
Failed on AISI-SAE 4037


steel,AZoM-Successful
str,bool
"""AISI-SAE 1080""",false
"""AISI-SAE 1042""",false
"""AISI 1049""",false
"""AISI-SAE 9254""",false
"""AISI 4047""",false
"""AISI 52100""",false
"""AISI-SAE 1335""",false
"""AISI-SAE 4037""",false


## Convert to dataframes and save

Checkpoint for continued work in this file so the scraping does not need to be done every time.

In [37]:
ttCdf_temp = pl.DataFrame(list_of_scraped, orient='row')
# rename columns using first row
# https://stackoverflow.com/questions/75187317/how-to-rename-column-names-with-first-row-in-polars
# [1:] removes the first row that replaced the column names 
ttCdf_temp = ttCdf_temp.rename(ttCdf_temp.head(1).to_dicts().pop())[1:]
ttCdf_temp.tail(3)

steel,pr,em,em_unit,ys,ys_unit,uts,uts_unit
str,str,str,str,str,str,str,str
"""AISI-SAE 4340""","""0.27-0.30""","""190-210""","""GPa""","""470""","""MPa""","""745""","""MPa"""
"""AISI-SAE 6150""","""0.27 – 0.30""","""190-210""","""GPa""","""415""","""MPa""","""670""","""MPa"""
"""AISI 6145""","""0.27-030""","""201-209""","""GPa""","""1165""","""MPa""","""1213""","""MPa"""


In [38]:
df_steel_properties = df_steel_properties.join(ttCdf_temp, on='steel', how='inner')
df_steel_properties.write_csv(f'{resources_path}/scraped_properties.csv')
df_steel_properties


steel,density,units_density,pr,em,em_unit,ys,ys_unit,uts,uts_unit
str,str,str,str,str,str,str,str,str,str
"""AISI-SAE 1038""","""7.845""","""g/cm3""","""0.27-0.30""","""190-210""","""GPa""","""485""","""MPa""","""570""","""MPa"""
"""AISI-SAE 1065""","""7.85""","""g/cm3""","""0.27-0.30""","""200""","""GPa""","""490""","""MPa""","""635""","""MPa"""
"""AISI-SAE 5160""","""7.85""","""g/cm3""","""0.27 – 0.30""","""190-210""","""GPa""","""275""","""MPa""","""724""","""MPa"""
"""AISI 1090""","""7.85""","""g/cm3""","""0.27-0.30""","""190-210""","""GPa""","""540""","""MPa""","""696""","""MPa"""
"""AISI 1055""","""7.85""","""g/cm3""","""0.27-0.30""","""190-210""","""GPa""","""560""","""MPa""","""660""","""MPa"""
…,…,…,…,…,…,…,…,…,…
"""AISI-SAE 1035""","""7.85""","""g/cm3""","""0.27-0.30""","""190-210""","""GPa""","""370""","""MPa""","""585""","""MPa"""
"""AISI-SAE 1050""","""7.85""","""g/cm3""","""0.27-0.30""","""190-210""","""GPa""","""580""","""MPa""","""690""","""MPa"""
"""AISI-SAE 4340""","""7.85""","""g/cm3""","""0.27-0.30""","""190-210""","""GPa""","""470""","""MPa""","""745""","""MPa"""


In [39]:
s_AZOM_unsuccessful = df_AZoM_successfull.filter(pl.col('AZoM-Successful')==False)['steel']
pl.DataFrame(s_AZOM_unsuccessful).write_csv(f'{resources_path}/MIF_search.csv')

# Process units from scraped AZoM data

Ensure all units are base units and the same down each column

In [40]:
data_path = f"{resources_path}/scraped_properties.csv"
df_scraped = pl.read_csv(data_path)
df_scraped.glimpse()

Rows: 21
Columns: 10
$ steel         <str> 'AISI-SAE 1038', 'AISI-SAE 1065', 'AISI-SAE 5160', 'AISI 1090', 'AISI 1055', 'AISI 1030', 'AISI-SAE 5140', 'AISI-SAE 4140', 'AISI 4640', 'AISI-SAE 1045'
$ density       <str> '7.845', '7.85', '7.85', '7.85', '7.85', '7.85', '7.85', '7.85', '7.75', '7.87'
$ units_density <str> 'g/cm3', 'g/cm3', 'g/cm3', 'g/cm3', 'g/cm3', 'g/cc', 'g/cm3', 'g/cm3', 'g/cm3', 'g/cm3'
$ pr            <str> '0.27-0.30', '0.27-0.30', '0.27 – 0.30', '0.27-0.30', '0.27-0.30', '0.27-0.30', '0.27-0.30', '0.27-0.30', '0.27-030', '0.29'
$ em            <str> '190-210', '200', '190-210', '190-210', '190-210', '190-210', '190-210', '190-210', '201-209', '200'
$ em_unit       <str> 'GPa', 'GPa', 'GPa', 'GPa', 'GPa', 'GPa', 'GPa', 'GPa', 'GPa', 'GPa'
$ ys            <i64> 485, 490, 275, 540, 560, 440, 295, 415, 1103, 450
$ ys_unit       <str> 'MPa', 'MPa', 'MPa', 'MPa', 'MPa', 'MPa', 'MPa', 'MPa', 'MPa', 'MPa'
$ uts           <i64> 570, 635, 724, 696, 660, 525, 570, 655, 1276, 

In [41]:
df_scraped.select(['units_density', 'em_unit', "ys_unit", "uts_unit"]).unique()

units_density,em_unit,ys_unit,uts_unit
str,str,str,str
"""g/""","""GPa""","""MPa""","""MPa"""
"""g/cc""","""GPa""","""MPa""","""MPa"""
"""g/cm3""","""GPa""","""MPa""","""MPa"""


## All units are valid and ready to be added to header and unit column removed

All units in units_density are a 1:1 conversion with g/cm3

g/mL (water) = g/cc = g/cm3 = grams per cubic centimeter

Converns:
* emdash and hyphens present
* AISI-SAE 1065 did not separate from it's decimal number.
* AISI-SAE 1026 indicatse g/ but was originally "g/ cm3"

In [42]:
df_scraped_unit_header = df_scraped.select(['steel', 
                                            'density', 
                                            'pr', 
                                            'em', 
                                            'ys',
                                            'uts']).rename({'density' : 'density_g_per_cm3',
                                                                        'em' : 'em_GPa',
                                                                        'ys' : 'ys_MPa',
                                                                        'uts' : 'uts_MPa'})

print(df_scraped_unit_header.shape)    
df_scraped_unit_header.head()

(21, 6)


steel,density_g_per_cm3,pr,em_GPa,ys_MPa,uts_MPa
str,str,str,str,i64,i64
"""AISI-SAE 1038""","""7.845""","""0.27-0.30""","""190-210""",485,570
"""AISI-SAE 1065""","""7.85""","""0.27-0.30""","""200""",490,635
"""AISI-SAE 5160""","""7.85""","""0.27 – 0.30""","""190-210""",275,724
"""AISI 1090""","""7.85""","""0.27-0.30""","""190-210""",540,696
"""AISI 1055""","""7.85""","""0.27-0.30""","""190-210""",560,660


In [43]:
df_scraped_unit_header.describe()

statistic,steel,density_g_per_cm3,pr,em_GPa,ys_MPa,uts_MPa
str,str,str,str,str,f64,f64
"""count""","""21""","""21""","""21""","""21""",21.0,21.0
"""null_count""","""0""","""0""","""0""","""0""",0.0,0.0
"""mean""",null,null,null,null,508.47619,680.190476
"""std""",null,null,null,null,223.223569,200.888183
"""min""","""AISI 1030""","""7.7-8.03""","""0.27 – 0.30""","""190-210""",275.0,490.0
"""25%""",null,null,null,null,415.0,570.0
"""50%""",null,null,null,null,450.0,650.0
"""75%""",null,null,null,null,525.0,690.0
"""max""","""AISI-SAE 6150""","""7.87""","""0.29""","""201-209""",1165.0,1276.0


## Process ranges to an average value

### Helper functions for mapping

In [44]:
df_scraped_clean = df_scraped_unit_header.clone()


In [45]:
# string to number conversion that is aware all values arefrom sklearn.cluster import KMeans, AgglomerativeClustering
def strip_string(string):
    return string.strip()

def if_no_decimal_point(num):
    #If leading 0 followed by number then replace leading 0 with "0."
    regex = r"^0\d"
    if re.search(regex, num):
        #replace the 
        num = re.sub("^0", "0.", num)
    return np.float64(num)

def process_ranged_val_cleaned(col, df=df_scraped_clean):
    # AISI-SAE 5160 and AISI-SAE 6150 use a different dash than the others for the pr range
    # Discovered when 2 nulls showed up after processing
        # mean could not process the string and resulted in a null
    s_same_dashes = df[col].str.replace('–', '-')
    # Split the range froma string into two strings of numbers
    s_split = s_same_dashes.str.split('-')
    # Convert these decimal points to numbers 
        #aware that some decimal values are formatted 0## with no .
    s_numeric_l = s_split.map_elements(lambda x: list(map(if_no_decimal_point, x)))
    # take middle of the two values
    return s_numeric_l.list.mean()

# Process Units 

In [46]:
# Split the range froma string into two strings of numbers
s_split = df_scraped_clean['density_g_per_cm3'].str.split('-')
# Convert these decimal points to numbers 
s_numeric_l = s_split.map_elements(lambda x: list(map(if_no_decimal_point, x)))
# take middle of the two values
s_mean_d = s_numeric_l.list.mean()

/tmp/ipykernel_34738/1347279945.py:4: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
  s_numeric_l = s_split.map_elements(lambda x: list(map(if_no_decimal_point, x)))


In [47]:
s_mean_p = process_ranged_val_cleaned("pr")
s_mean_em = process_ranged_val_cleaned("em_GPa")

/tmp/ipykernel_34738/2290863345.py:22: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
  s_numeric_l = s_split.map_elements(lambda x: list(map(if_no_decimal_point, x)))


In [48]:
df_AZoM_clean = df_scraped_clean.with_columns(poissons_ratio = s_mean_p, 
                                         density_g_per_cm3 = s_mean_d, 
                                         elastic_modulus_GPa = s_mean_em,
                                         yield_strength_MPa = df_scraped_unit_header['ys_MPa'],
                                         ultimate_strength_MPA = df_scraped_unit_header['uts_MPa'])\
                                            .select(["steel",
                                                     "density_g_per_cm3",
                                                     "elastic_modulus_GPa",
                                                     "yield_strength_MPa",
                                                     "ultimate_strength_MPA"])
df_AZoM_clean.describe()

statistic,steel,density_g_per_cm3,elastic_modulus_GPa,yield_strength_MPa,ultimate_strength_MPA
str,str,f64,f64,f64,f64
"""count""","""21""",21.0,21.0,21.0,21.0
"""null_count""","""0""",0.0,0.0,0.0,0.0
"""mean""",null,7.842048,200.47619,508.47619,680.190476
"""std""",null,0.031154,1.503963,223.223569,200.888183
"""min""","""AISI 1030""",7.75,200.0,275.0,490.0
"""25%""",null,7.85,200.0,415.0,570.0
"""50%""",null,7.85,200.0,450.0,650.0
"""75%""",null,7.85,200.0,525.0,690.0
"""max""","""AISI-SAE 6150""",7.87,205.0,1165.0,1276.0


# Drop columns that do not add information

Poisson's Ratio, Density, and Elastic Modulus are singletons. They do not add information.

In [49]:
df_AZoM_clean = df_AZoM_clean.select(['steel',
                                    "yield_strength_MPa",
                                    "ultimate_strength_MPA"])
df_AZoM_clean.describe()

statistic,steel,yield_strength_MPa,ultimate_strength_MPA
str,str,f64,f64
"""count""","""21""",21.0,21.0
"""null_count""","""0""",0.0,0.0
"""mean""",null,508.47619,680.190476
"""std""",null,223.223569,200.888183
"""min""","""AISI 1030""",275.0,490.0
"""25%""",null,415.0,570.0
"""50%""",null,450.0,650.0
"""75%""",null,525.0,690.0
"""max""","""AISI-SAE 6150""",1165.0,1276.0


In [50]:
df_AZoM_clean.write_csv(f'{resources_path}/scraped_properties.csv')

## Scraping MakeItFrom.com
DuckDuckGo search used since the steels of question are split in two categories and MakeItFrom does not have a search function.

In [51]:
# https://www.makeitfrom.com/material-group/Wrought-Alloy-Steel-SAE-AISI
# OR
# https://www.makeitfrom.com/material-group/Wrought-Carbon-Or-Non-Alloy-Steel
base_url = "https://www.makeitfrom.com"
search_url = "https://duckduckgo.com/?hps=1&q="
search_suffix = "+site%3Amakeitfrom.com&atb=v427-1&ia=web"

In [52]:
s_AZOM_unsuccessful

steel
str
"""AISI-SAE 1080"""
"""AISI-SAE 1042"""
"""AISI 1049"""
"""AISI-SAE 9254"""
"""AISI 4047"""
"""AISI 52100"""
"""AISI-SAE 1335"""
"""AISI-SAE 4037"""


In [53]:
def handle_ranges(values_list):
    if values_list[1] == 'to':
        # map the range of values to a list of integers, then take the mean of it.
        value = np.mean(list(map(int, values_list[0:3:2])))
        units = values_list[3]
    else:
        value = np.float64(values_list[0])
        units = values_list[1]
    return value, units

In [54]:
def get_row(soup):
    mech_props = soup.find_all('div', class_='mech')
    # -2 is a magic number
    # the last 2 values are always tensile strength
    uts = 'no_label'
    uts_units = 'no_units'
    ys = 'no_label'
    ys_units = 'no_units'

    for div in mech_props[-2:]:
        ps = div.find_all('p')
        # get the property name
        label = ps[0].text
        # get the string of values and units and split it
        values = ps[1].text.split(' ')
        # Grab the metric value 
        value, units = handle_ranges(values)
        if 'uts' in label.lower():
            uts = value
            uts_units = units
        elif 'yield' in label.lower():
            ys = value
            ys_units = units
    row = (steel, ys, ys_units, uts, uts_units)
    return row

In [55]:
def handle_cold_drawn():
    # Browser is already on steel page
    soup = BeautifulSoup(browser.html, 'html.parser')
    cold_drawn_present = True
    steel_page = soup.find('div', class_='split links break-mid')
    # https://stackoverflow.com/questions/33404049/navigation-with-beautifulsoup
    
    cold_drawn = steel_page.find('a', string= lambda text: text \
                            #cold drawn matches values from AZoM for steels that are on both
                                and "Cold Drawn" in text\
                                # not "and" removed other processing in addtional to cold drawn if applicable
                                and not "and" in text) 
    try: 
        cold_drawn_page = cold_drawn.get('href')
        time.sleep(0.15)
        browser.visit(f'{base_url}{cold_drawn_page}')
    except AttributeError:
        # do not go to the cold drawn page if not present
        pass    

In [56]:

list_of_scraped = [("steel", "ys", "ys_unit", "uts", "uts_unit")] 
for steel in s_AZOM_unsuccessful:
    print("Working on:", steel)
    #search MIF for the steel
    time.sleep(0.15)
    browser.visit(f"{search_url}{steel}{search_suffix}")
    soup_search = BeautifulSoup(browser.html, 'html.parser')
    steel_link = soup_search.find('a',{'data-testid': "result-extras-url-link"}).get('href')
    
    time.sleep(0.15)
    browser.visit(steel_link)
    # handle cold drawn will land us on the desired steel page
    handle_cold_drawn()
    soup_results = BeautifulSoup(browser.html, "html.parser")
    list_of_scraped.append(get_row(soup_results))

browser.quit()

list_of_scraped

Working on: AISI-SAE 1080
Working on: AISI-SAE 1042
Working on: AISI 1049
Working on: AISI-SAE 9254
Working on: AISI 4047
Working on: AISI 52100
Working on: AISI-SAE 1335
Working on: AISI-SAE 4037


[('steel', 'ys', 'ys_unit', 'uts', 'uts_unit'),
 ('AISI-SAE 1080', 535.0, 'MPa', 820.0, 'MPa'),
 ('AISI-SAE 1042', 580.0, 'MPa', 700.0, 'MPa'),
 ('AISI 1049', 640.0, 'MPa', 750.0, 'MPa'),
 ('AISI-SAE 9254', 410.0, 'MPa', 660.0, 'MPa'),
 ('AISI 4047', 310.0, 'MPa', 580.0, 'MPa'),
 ('AISI 52100', 460.0, 'MPa', 1300.0, 'MPa'),
 ('AISI-SAE 1335', 300.0, 'MPa', 550.0, 'MPa'),
 ('AISI-SAE 4037', 290.0, 'MPa', 540.0, 'MPa')]

In [57]:
df_MIF = pl.DataFrame(list_of_scraped, orient='row')
df_MIF = df_MIF.rename(df_MIF.head(1).to_dicts().pop())[1:]
df_MIF

steel,ys,ys_unit,uts,uts_unit
str,str,str,str,str
"""AISI-SAE 1080""","""535""","""MPa""","""820""","""MPa"""
"""AISI-SAE 1042""","""580""","""MPa""","""700""","""MPa"""
"""AISI 1049""","""640""","""MPa""","""750""","""MPa"""
"""AISI-SAE 9254""","""410""","""MPa""","""660""","""MPa"""
"""AISI 4047""","""310""","""MPa""","""580""","""MPa"""
"""AISI 52100""","""460""","""MPa""","""1300""","""MPa"""
"""AISI-SAE 1335""","""300""","""MPa""","""550""","""MPa"""
"""AISI-SAE 4037""","""290""","""MPa""","""540""","""MPa"""


In [58]:
df_MIF_clean = df_MIF.select(['steel', 'ys', 'uts']).rename({'ys' : 'yield_strength_MPa',
                                                             'uts' : 'ultimate_strength_MPA'})
df_MIF_clean

steel,yield_strength_MPa,ultimate_strength_MPA
str,str,str
"""AISI-SAE 1080""","""535""","""820"""
"""AISI-SAE 1042""","""580""","""700"""
"""AISI 1049""","""640""","""750"""
"""AISI-SAE 9254""","""410""","""660"""
"""AISI 4047""","""310""","""580"""
"""AISI 52100""","""460""","""1300"""
"""AISI-SAE 1335""","""300""","""550"""
"""AISI-SAE 4037""","""290""","""540"""


In [59]:
df_AZoM_clean

steel,yield_strength_MPa,ultimate_strength_MPA
str,i64,i64
"""AISI-SAE 1038""",485,570
"""AISI-SAE 1065""",490,635
"""AISI-SAE 5160""",275,724
"""AISI 1090""",540,696
"""AISI 1055""",560,660
…,…,…
"""AISI-SAE 1035""",370,585
"""AISI-SAE 1050""",580,690
"""AISI-SAE 4340""",470,745


In [60]:
df_complete_props = pl.concat([df_AZoM_clean, df_MIF_clean], how='vertical_relaxed')
df_complete_props = df_complete_props.with_columns(pl.col('yield_strength_MPa').cast(pl.Int64),
                                       pl.col('ultimate_strength_MPA').cast(pl.Int64))
df_complete_props

steel,yield_strength_MPa,ultimate_strength_MPA
str,i64,i64
"""AISI-SAE 1038""",485,570
"""AISI-SAE 1065""",490,635
"""AISI-SAE 5160""",275,724
"""AISI 1090""",540,696
"""AISI 1055""",560,660
…,…,…
"""AISI-SAE 9254""",410,660
"""AISI 4047""",310,580
"""AISI 52100""",460,1300


In [61]:
df_complete_props.write_csv(f'{resources_path}/scraped_properties.csv')

# Join properties data with tempering data
Inner join to exclude the steels that are not searchable with the given information, or that do not conform to any known and accessable standards.

1286 rows after join

In [62]:
df_complete_props = df_complete_props.rename({'steel':'searchable'})
data = series_searchable_steel.join(df_complete_props, how="inner", on='searchable')

In [63]:
data.shape

(1286, 20)

# Select columns that have relevant data for the analysis

Exclude:
* Source data - does not provide information about the problem
* initial hardness - very incomplete data and no accurate way to fill it
* searchable -  contains more information than searchable, but is likely relipcated in elemental
  * Searchable contains fewer values, this grouping was used to scrape the strength values and contains no additional information
  * steel_type contains steel denotation from the papers. The informationa in this column is replicated in the elemental composition columns
  * PCA will handle this


In [64]:
df_relevent = data.drop(['source', 'searchable', 'initial_hardness_post_quenching_HRC'])
df_relevent.columns

['steel_type',
 'tempering_time_s',
 'tempering_temperature_C',
 'C',
 'Mn',
 'P',
 'S',
 'Si',
 'Ni',
 'Cr',
 'Mo',
 'V',
 'Al',
 'Cu',
 'final_hardness_post_tempering_HRC',
 'yield_strength_MPa',
 'ultimate_strength_MPA']

# Preprocessing the data for machine learning

In [65]:
target_columns = [ 'tempering_time_s', 'tempering_temperature_C']

X_prepre = df_relevent.drop(target_columns)

y_prepre = df_relevent.select(target_columns)

In [66]:
y_prepre.head(3)

tempering_time_s,tempering_temperature_C
i64,f64
600,204.4
600,260.0
600,315.6


# Preprocess target values

All are converted to Celcius but several are very near together because theyw ere originally Farenheight.

eg 200 C is negligably near 204.4 C which equals 400 F.

55 C ranges were picked, as it makes 11 whole number bins from 100 to 705


## Bin tempering values by 55 C

38 C was considered (100 F) but some values were indistinct. 
55 C is a whole number value that results in a whole number of bins.

It results in 11 bins. 

This is the same number of bins when we bin tempering time <= 5 minutes

In [67]:
tempering_bins = range(155, 705, 55)
list(tempering_bins)

[155, 210, 265, 320, 375, 430, 485, 540, 595, 650]

In [68]:
labels = ['100', '200', '250', '300', '350', '400', '450', '500', '550', '600', '700']

In [69]:
#binned_temperting_tempC = 
tempering_tempC_binned = y_prepre['tempering_temperature_C'].cut(tempering_bins, labels=labels)
tempering_tempC_binned


tempering_temperature_C
cat
"""200"""
"""250"""
"""300"""
"""350"""
"""400"""
…
"""300"""
"""400"""
"""500"""


In [70]:
y = pl.DataFrame({'tempering_time_s_proc': y_prepre['tempering_time_s'],
    #'tempering_time_s_proc': tts_bin_string,
                   'tempering_temperature_C_proc': tempering_tempC_binned})
y

tempering_time_s_proc,tempering_temperature_C_proc
i64,cat
600,"""200"""
600,"""250"""
600,"""300"""
600,"""350"""
600,"""400"""
…,…
86400,"""300"""
86400,"""400"""
86400,"""500"""


# Preprocess data values

In [71]:
print(X_prepre.shape)
X_prepre.head(3)

(1286, 15)


steel_type,C,Mn,P,S,Si,Ni,Cr,Mo,V,Al,Cu,final_hardness_post_tempering_HRC,yield_strength_MPa,ultimate_strength_MPA
str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,i64,i64
"""AISI-SAE 1026""",0.25,0.79,0.012,0.026,0.11,0.0,0.0,0.0,0.0,0.0,0.0,50.6,415,490
"""AISI-SAE 1026""",0.25,0.79,0.012,0.026,0.11,0.0,0.0,0.0,0.0,0.0,0.0,48.3,415,490
"""AISI-SAE 1026""",0.25,0.79,0.012,0.026,0.11,0.0,0.0,0.0,0.0,0.0,0.0,43.7,415,490


# Encode and scale variables
Most columns are numeric and range from 0.0# to 1000 so scaling is necessary
steel names are categorical and would benefit from one-hot encoding.

## One hot encode steel types

Encoding chosen because this is a categorical for a given material and testing suite

In [72]:
to_scale = X_prepre.drop('steel_type')
to_onehot = X_prepre.select('steel_type')

In [73]:
to_onehot

steel_type
str
"""AISI-SAE 1026"""
"""AISI-SAE 1026"""
"""AISI-SAE 1026"""
"""AISI-SAE 1026"""
"""AISI-SAE 1026"""
…
"""1,15%C - plain carbon steel"""
"""1,15%C - plain carbon steel"""
"""1,15%C - plain carbon steel"""


In [74]:
# create the transformers
ohe = OneHotEncoder(handle_unknown='ignore')

oh_transformed = ohe.fit_transform(to_onehot)

# get the transformed column names
ohe_names = ohe.get_feature_names_out()

# 4 stack exchange methods tried. Then I made this one up ¯\_(ツ)_/¯
# Combinated of recenly changed API and polars I think, no way to rename all columns like in pandas.
columns = [f'column_{x}' for x in range(0, 30)]
rename_dict = dict(zip(columns, ohe_names))

df_ohe_encoded = pl.DataFrame(oh_transformed.toarray()).rename(rename_dict)
df_ohe_encoded.tail(3)

"steel_type_0,31%C - plain carbon steel","steel_type_0,56%C - plain carbon steel","steel_type_0,74%C - plain carbon steel","steel_type_0,89%C - plain carbon steel","steel_type_0,98%C - plain carbon steel","steel_type_1,15%C - plain carbon steel",steel_type_AISI-SAE 1026,steel_type_AISI-SAE 1030,steel_type_AISI-SAE 1035,steel_type_AISI-SAE 1038,steel_type_AISI-SAE 1040,steel_type_AISI-SAE 1042,steel_type_AISI-SAE 1045,steel_type_AISI-SAE 1049,steel_type_AISI-SAE 1050,steel_type_AISI-SAE 1065,steel_type_AISI-SAE 1080,steel_type_AISI-SAE 1335,steel_type_AISI-SAE 4027,steel_type_AISI-SAE 4037,steel_type_AISI-SAE 4047,steel_type_AISI-SAE 4140,steel_type_AISI-SAE 4340,steel_type_AISI-SAE 4640,steel_type_AISI-SAE 5140,steel_type_AISI-SAE 5160,steel_type_AISI-SAE 6145,steel_type_AISI-SAE 6150,steel_type_AISI-SAE 9264,steel_type_AISI-SAE E52100
f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Scale numerical columns 
This will ensure the small elemental composition values are not overshadowed by the large strength properties 

In [75]:
print(type(to_scale))
to_scale.tail(3)

<class 'polars.dataframe.frame.DataFrame'>


C,Mn,P,S,Si,Ni,Cr,Mo,V,Al,Cu,final_hardness_post_tempering_HRC,yield_strength_MPa,ultimate_strength_MPA
f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,i64,i64
1.15,0.58,0.012,0.021,0.09,0.0,0.01,0.0,0.0,0.0,0.0,32.0,525,685
1.15,0.58,0.012,0.021,0.09,0.0,0.01,0.0,0.0,0.0,0.0,23.0,525,685
1.15,0.58,0.012,0.021,0.09,0.0,0.01,0.0,0.0,0.0,0.0,4.5,525,685


In [76]:
scaler = StandardScaler()

X_scaler = scaler.fit(to_scale)

scaled = scaler.transform(to_scale)

scaler_names = scaler.get_feature_names_out()
columns = [f'column_{x}' for x in range(0, 14)]
rename_dict = dict(zip(columns, scaler_names))

df_scaled = pl.DataFrame(scaled).rename(rename_dict)
df_scaled.tail(3)


C,Mn,P,S,Si,Ni,Cr,Mo,V,Al,Cu,final_hardness_post_tempering_HRC,yield_strength_MPa,ultimate_strength_MPA
f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
2.69532,-0.618793,-0.60703,-0.394811,-0.591857,-0.392117,-0.791743,-0.638219,-0.201129,0.0,-0.337311,-0.682819,0.188703,-0.105424
2.69532,-0.618793,-0.60703,-0.394811,-0.591857,-0.392117,-0.791743,-0.638219,-0.201129,0.0,-0.337311,-1.309217,0.188703,-0.105424
2.69532,-0.618793,-0.60703,-0.394811,-0.591857,-0.392117,-0.791743,-0.638219,-0.201129,0.0,-0.337311,-2.596813,0.188703,-0.105424


## Merge one hot encoded variables with scaled numeric variables

In [77]:
X = pl.concat([df_ohe_encoded, df_scaled], how='horizontal')
X

"steel_type_0,31%C - plain carbon steel","steel_type_0,56%C - plain carbon steel","steel_type_0,74%C - plain carbon steel","steel_type_0,89%C - plain carbon steel","steel_type_0,98%C - plain carbon steel","steel_type_1,15%C - plain carbon steel",steel_type_AISI-SAE 1026,steel_type_AISI-SAE 1030,steel_type_AISI-SAE 1035,steel_type_AISI-SAE 1038,steel_type_AISI-SAE 1040,steel_type_AISI-SAE 1042,steel_type_AISI-SAE 1045,steel_type_AISI-SAE 1049,steel_type_AISI-SAE 1050,steel_type_AISI-SAE 1065,steel_type_AISI-SAE 1080,steel_type_AISI-SAE 1335,steel_type_AISI-SAE 4027,steel_type_AISI-SAE 4037,steel_type_AISI-SAE 4047,steel_type_AISI-SAE 4140,steel_type_AISI-SAE 4340,steel_type_AISI-SAE 4640,steel_type_AISI-SAE 5140,steel_type_AISI-SAE 5160,steel_type_AISI-SAE 6145,steel_type_AISI-SAE 6150,steel_type_AISI-SAE 9264,steel_type_AISI-SAE E52100,C,Mn,P,S,Si,Ni,Cr,Mo,V,Al,Cu,final_hardness_post_tempering_HRC,yield_strength_MPa,ultimate_strength_MPA
f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.155898,0.167872,-0.60703,0.268364,-0.513467,-0.392117,-0.813526,-0.638219,-0.201129,0.0,-0.337311,0.611736,-0.342255,-1.009596
0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.155898,0.167872,-0.60703,0.268364,-0.513467,-0.392117,-0.813526,-0.638219,-0.201129,0.0,-0.337311,0.451657,-0.342255,-1.009596
0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.155898,0.167872,-0.60703,0.268364,-0.513467,-0.392117,-0.813526,-0.638219,-0.201129,0.0,-0.337311,0.131498,-0.342255,-1.009596
0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.155898,0.167872,-0.60703,0.268364,-0.513467,-0.392117,-0.813526,-0.638219,-0.201129,0.0,-0.337311,-0.091221,-0.342255,-1.009596
0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.155898,0.167872,-0.60703,0.268364,-0.513467,-0.392117,-0.813526,-0.638219,-0.201129,0.0,-0.337311,-0.31394,-0.342255,-1.009596
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.69532,-0.618793,-0.60703,-0.394811,-0.591857,-0.392117,-0.791743,-0.638219,-0.201129,0.0,-0.337311,0.604776,0.188703,-0.105424
0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.69532,-0.618793,-0.60703,-0.394811,-0.591857,-0.392117,-0.791743,-0.638219,-0.201129,0.0,-0.337311,0.013178,0.188703,-0.105424
0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.69532,-0.618793,-0.60703,-0.394811,-0.591857,-0.392117,-0.791743,-0.638219,-0.201129,0.0,-0.337311,-0.682819,0.188703,-0.105424


In [78]:
X.write_csv(f"{resources_path}/X_nopreproc.csv")
y.write_csv(f"{resources_path}/y_nopreproc.csv")


## Modelling for data preprocess

PCA modelling used to reduce X dimensions from 44 to 12 maintaining 95.7% of the explained variance.

KBinsDiscretizer used to bin imbalanced Y target. 15 bins, where half have less than 5 support reduced to 5 similarly supported bins

### PCA analysis

Introduction of two strength metrics certainly introduced multcolinearity, with eachother and with the elemental composision, some of whicha re indicators of strength.

PCA analysis will be done to alleiviate these factors.

In [79]:
X = pl.read_csv(f"{resources_path}/X_nopreproc.csv")

In [80]:
pca_explained_sum = {'x': [], 'pca_explained_var': []}
pca_explained_min = {'x': [], 'pca_explained_min_component': []}
for x in range(3,30):
    pca = PCA(n_components=x ,random_state=random_state)

    pca.fit(X)

    pca_explained_sum['x'].append(x)
    sum_explained = np.sum(pca.explained_variance_ratio_)
    pca_explained_sum['pca_explained_var'].append(sum_explained)

    pca_explained_min['x'].append(x)
    min_explained = np.min(pca.explained_variance_ratio_)
    pca_explained_min['pca_explained_min_component'].append(min_explained)

In [81]:
df_pca_explained = pl.DataFrame(pca_explained_sum)

### PCA component number determination

In [82]:
alt.Chart(df_pca_explained,
          title= "Total Explained Vairance vs Nubmer of PCA Features").mark_line().encode(
    alt.X('x:Q').title("PCA X"),
    alt.Y('pca_explained_var:Q')\
        .scale(domain=(0.5,1))\
            .title("Total explained variance")
)

alt.Chart(...)

### Value of increasing PCA components decreases between 10 and 15

In [83]:
df_pca_explained_min = pl.DataFrame(pca_explained_min)
df_pca_explained_min.filter(pl.col('x').le(15) & pl.col('x').ge(10))

x,pca_explained_min_component
i64,f64
10,0.026609
11,0.01642
12,0.012011
13,0.007145
14,0.003059
15,0.002843


### PCA component number 12 selected
Component 13 has half the explained ratio contribution as component 12, and is below 1%.

In [84]:
df_pca_explained.filter(pl.col("x").eq(12))

x,pca_explained_var
i64,f64
12,0.957246


In [85]:
pca = PCA(n_components= 12, random_state=random_state)

pca.fit(X)

X_pca = pca.transform(X)

df_pca = pl.DataFrame(X_pca)
df_pca.head(3)

column_0,column_1,column_2,column_3,column_4,column_5,column_6,column_7,column_8,column_9,column_10,column_11
f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
-1.321473,0.430856,0.153289,-0.283008,-0.555665,-0.439498,0.785101,-0.155914,-1.116482,-0.683583,0.08833,0.411546
-1.353472,0.404135,0.167922,-0.324673,-0.625215,-0.418999,0.672451,-0.158293,-1.059672,-0.66215,0.094859,0.398162
-1.41747,0.350694,0.197188,-0.408003,-0.764315,-0.377999,0.447151,-0.163052,-0.946051,-0.619285,0.107917,0.371395


In [86]:
df_pca.describe()

statistic,column_0,column_1,column_2,column_3,column_4,column_5,column_6,column_7,column_8,column_9,column_10,column_11
str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""count""",1286.0,1286.0,1286.0,1286.0,1286.0,1286.0,1286.0,1286.0,1286.0,1286.0,1286.0,1286.0
"""null_count""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"""mean""",-4.4202e-17,6.9065e-17,-3.3151e-17,-9.1166e-17,-2.2101e-17,-2.7626e-18,5.5252e-18,-4.2820e-17,2.4863e-17,-8.7022e-17,-5.6633e-17,-3.1770e-17
"""std""",1.671421,1.571462,1.326849,1.226593,1.109965,0.940957,0.877639,0.812326,0.727373,0.609722,0.478962,0.409636
"""min""",-3.155398,-3.056202,-2.522623,-2.921009,-3.084877,-1.296769,-3.894343,-1.608288,-1.596398,-1.087096,-0.975328,-1.093947
"""25%""",-1.251004,-0.987825,-0.68221,-0.715957,-0.713447,-0.517181,-0.591398,-0.530525,-0.449696,-0.461062,-0.292013,-0.349081
"""50%""",-0.086811,-0.038677,-0.081856,-0.107492,0.041456,-0.126058,0.120074,-0.081411,-0.056779,-0.068896,-0.018839,0.093058
"""75%""",0.655301,0.973602,0.509383,0.799596,0.654078,0.302718,0.593107,0.522618,0.291119,0.336484,0.23024,0.317859
"""max""",4.648756,3.535733,4.2696,3.574884,4.379859,4.380695,2.405554,1.521652,2.622772,1.750866,1.039316,0.698489


## Check data balance before train-test-split

In [87]:
y = pl.read_csv(f"{resources_path}/y_nopreproc.csv", schema=pl.Schema({"tempering_time_s_proc":pl.String,
                                                             "tempering_temperature_C_proc":pl.String}))
y.columns

['tempering_time_s_proc', 'tempering_temperature_C_proc']

In [88]:
for col in ['tempering_time_s_proc', 'tempering_temperature_C_proc']:
    temp = y.select(pl.col(col)\
                    .value_counts(sort=True, name='n'))\
                        .unnest(col)\
                            .with_columns(pl.col(col).cast(pl.Int32))
    print(temp)
 


shape: (15, 2)
┌───────────────────────┬─────┐
│ tempering_time_s_proc ┆ n   │
│ ---                   ┆ --- │
│ i32                   ┆ u32 │
╞═══════════════════════╪═════╡
│ 3600                  ┆ 237 │
│ 86400                 ┆ 206 │
│ 14400                 ┆ 130 │
│ 900                   ┆ 120 │
│ 600                   ┆ 108 │
│ …                     ┆ …   │
│ 57600                 ┆ 30  │
│ 115200                ┆ 30  │
│ 40                    ┆ 24  │
│ 120                   ┆ 24  │
│ 300                   ┆ 24  │
└───────────────────────┴─────┘
shape: (11, 2)
┌──────────────────────────────┬─────┐
│ tempering_temperature_C_proc ┆ n   │
│ ---                          ┆ --- │
│ i32                          ┆ u32 │
╞══════════════════════════════╪═════╡
│ 400                          ┆ 170 │
│ 500                          ┆ 167 │
│ 600                          ┆ 164 │
│ 300                          ┆ 145 │
│ 200                          ┆ 141 │
│ …                            ┆ …  

## Unbalanced targets needs processing

Some classifications has 200+ support but many have 50 or less in both metrics. (above)

Binning through an algorithm will be used to allieviate this

Before binning the f1 score scores were exceptionally low and support scores were very imbalanced. 

![before_balancing.png](images/before_balancing.png)
![after_balancing.png](images/after_balancing.png)

In [89]:
# https://imbalanced-learn.org/stable/common_pitfalls.html
t_temp_proc = y.select('tempering_temperature_C_proc')
y

tempering_time_s_proc,tempering_temperature_C_proc
str,str
"""600""","""200"""
"""600""","""250"""
"""600""","""300"""
"""600""","""350"""
"""600""","""400"""
…,…
"""86400""","""300"""
"""86400""","""400"""
"""86400""","""500"""


In [90]:
# vshort short med long vlong?
# Kbinsdescritizer
# [0-10min),[10min-60min), [60min-150min),[2.5h-8h), [8h+)   
y = y.with_columns(pl.col('tempering_time_s_proc').cast(pl.Int32))
bin_this = y['tempering_time_s_proc']

In [91]:
to_bin = bin_this.reshape((-1,1))
binner = KBinsDiscretizer(n_bins = 5, encode='ordinal', strategy='quantile')
binner.fit(to_bin)
binned = binner.transform(to_bin)
bin_labels = []

edges = binner.bin_edges_[0]
for i in range(len(edges)-1):
    #TODO: address
    label = f"< {edges [i + 1] / 60} min"
    label = edges [i + 1] / 60
    bin_labels.append(int(label))
bin_labels


[5, 60, 150, 960, 1920]

In [92]:
dict_label_bins = dict(zip(['0.0','1.0','2.0','3.0','4.0'], bin_labels))
dict_label_bins

{'0.0': 5, '1.0': 60, '2.0': 150, '3.0': 960, '4.0': 1920}

In [93]:
to_label = pl.DataFrame(binned).with_columns(pl.col('column_0')\
                                             .alias('tempering_time_s_binned'))\
                                                .select('tempering_time_s_binned')
y_time_binned = to_label.with_columns(pl.col('tempering_time_s_binned').cast(pl.String)\
                                      .replace(dict_label_bins))

In [94]:
y = pl.concat([y_time_binned, t_temp_proc], how='horizontal')
y

tempering_time_s_binned,tempering_temperature_C_proc
str,str
"""60""","""200"""
"""60""","""250"""
"""60""","""300"""
"""60""","""350"""
"""60""","""400"""
…,…
"""1920""","""300"""
"""1920""","""400"""
"""1920""","""500"""


In [95]:
df_pca.write_csv(f'{resources_path}/df_X.csv')
y.write_csv(f'{resources_path}/df_y.csv')


# Modelling

In [96]:
X = pl.read_csv(f'{resources_path}/df_X.csv')
y = pl.read_csv(f'{resources_path}/df_y.csv')

In [97]:
X = df_pca.to_numpy()
y = y.to_numpy()

## Test train split

In [99]:
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    random_state=random_state, 
                                                    stratify=y)

## Random ExtraTreesClassifier Model

Chosen as a multiclass multilabel classifier that works by default, without the use of MultiOutputClassifier.

### This naive model does not perform well.

Target Feature|Accuracy|f1 Score
---|---|---
TimeTarget|15.5%|29.7%
TempTarget|31.4%|15.9%

In [100]:
et_classifier = ExtraTreesClassifier(n_estimators=10, 
                                      criterion='entropy',
                                      random_state=random_state)

et_classifier.fit(X_train, y_train)


ExtraTreesClassifier(criterion='entropy', n_estimators=10,
                     random_state=2112250415)

In [101]:
y_pred = et_classifier.predict(X_test)


In [102]:
labels = []
target_prefixes = ["ttime_", "ttemp_"]
target_units = ['_min', '_C']

for i in range(y_test.shape[1]):
    prefixed = list(map(lambda x: f"{target_prefixes[i]}{x}{target_units[i]}", et_classifier.classes_[i]))
    labels.append(prefixed)

In [103]:
classification_reports = []
for i in range(y_test.shape[1]):
    cr = classification_report(y_test[:, i], y_pred[:, i],  output_dict=True)#target_names=labels[i],)
    classification_reports.append(cr)

In [104]:
# polars does not parse the dicts correctly
cr_rf_ttime = pd.DataFrame(classification_reports[0]).transpose()
cr_rf_ttemp = pd.DataFrame(classification_reports[1]).transpose()

def fix_classification_report(pd_df):
    pl_df = pl.DataFrame(pd_df.reset_index())\
        .filter(pl.col('index').str.contains('avg|acc').not_())\
        .with_columns(pl.col('index').cast(pl.Int32))\
        .sort('index', descending=False)
    return pl_df

pl_cr_rf_ttime = fix_classification_report(cr_rf_ttime)

pl_cr_rf_ttemp = fix_classification_report(cr_rf_ttemp)

# display 2 DF inline https://softhints.com/display-two-pandas-dataframes-side-by-side-jupyter-notebook/
cr_rf_ttime_styler = cr_rf_ttime.style.set_table_attributes("style='display:inline'")\
    .set_caption('Tempering Time')
cr_rf_ttemp_styler = cr_rf_ttemp.style.set_table_attributes("style='display:inline'")\
    .set_caption('Tempering Temperature')

space = "\xa0" * 10
display_html(cr_rf_ttemp_styler._repr_html_() + space + cr_rf_ttime_styler._repr_html_(), raw=True)

,precision,recall,f1-score,support
100,0.555556,0.517241,0.535714,29.000000
200,0.416667,0.416667,0.416667,36.000000
250,0.151515,0.178571,0.163934,28.000000
300,0.400000,0.444444,0.421053,36.000000
350,0.230769,0.250000,0.240000,12.000000
400,0.375000,0.279070,0.320000,43.000000
450,0.153846,0.153846,0.153846,13.000000
500,0.235294,0.285714,0.258065,42.000000
550,0.076923,0.105263,0.088889,19.000000
600,0.354839,0.275000,0.309859,40.000000


In [105]:

rf_cr_ttemp_supp_plot = alt.Chart(pl_cr_rf_ttemp)\
    .mark_bar().encode(
        alt.X('index:O').title('Tempering Temperature (C)'),
        alt.Y('support'),
        alt.Color('f1-score')
        )

rf_cr_ttime_supp_plot = alt.Chart(pl_cr_rf_ttime)\
    .mark_bar().encode(
        alt.X('index:O').title('Tempering Time (< min)'),
        alt.Y('support'),
        alt.Color('f1-score')
        )
rf_cr_ttemp_supp_plot

alt.concat(rf_cr_ttemp_supp_plot, rf_cr_ttime_supp_plot).properties(title='After Balancing Tempering Time')

alt.ConcatChart(...)

## ExtraTreesClassifier has very poor accuracy

15% for tempering time and 31% for tempering temperature

Moving to a Deep Neural Network model to try and improve performance. 

# Deep Neural Network Model

In [106]:
print("Xdim", X.shape[1],"| ydim", y.shape[1])

Xdim 12 | ydim 2


In [107]:

input_layer = Input(input_dim=12)
shared_layer = Dense(64, activation='relu')(input_layer)
shared_layer = Dense(64, activation='relu')(input_layer)
shared_layer = Dense(64, activation='relu')(input_layer)

# Output layers for each label
output1 = Dense(y_test[0].shape[1], activation='softmax', name='Tempering Time')(shared_layer)
output2 = Dense(y_test[1].  shape[1], activation='softmax', name='Tempering Temperaure')(shared_layer)

nn_model = Sequential()
nn_model.add()

nn_model.compile(optimizer='adam',
              loss={'label1_output': 'categorical_crossentropy', 'label2_output': 'categorical_crossentropy'},
              metrics=['accuracy'])

TypeError: Input() got an unexpected keyword argument 'input_dim'